## RNN
*The code below leverages an RNN model to forecast overall demand for the next month. However, since TensorFlow could not be imported in the local Anaconda environment due to compatibility issues, the implementation was carried out on Google Colab. The link is included below.*
<https://colab.research.google.com/drive/1Ufjzcy6uzZdB-dhK_tsBogcgRnMEm7C5?usp=sharing>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [ ]:
df["demand"] = df["unitprice"] * df["quantity"]
df['orderdate'] = pd.to_datetime(df['orderdate'])
df.set_index('orderdate', inplace=True)
df_monthly_demand = df['demand'].resample('M').sum()

print(df_monthly_demand.head())

In [ ]:
# Prepare data for RNN
data = df_monthly_demand.values.reshape(-1, 1)

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Create sequences for RNN
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12 # Adjust sequence length for monthly data, e.g., 12 for a year
X, y = create_sequences(data_scaled, seq_length)

# Split data into training and testing sets
train_size = int(len(X) * 0.8) # 80% for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
# Train the RNN model
history = model.fit(X_train, y_train, epochs=100, batch_size=1, validation_split=0.2) # Adjust epochs and batch_size for monthly data

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions and actual values
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)


In [ ]:
# Visualize the results
plt.figure(figsize=(12, 6))
plt.plot(df_monthly_demand.index[-len(y_test):], y_test_actual, label='Actual Demand')
plt.plot(df_monthly_demand.index[-len(y_test):], predictions.flatten(), label='Predicted Demand') # Flatten predictions for plotting
plt.title('RNN Demand forecast')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.show()